## Overriding default values

**A demo of two-level default and overriding.**

Say we have several "plates" and each we have several "wells" and we want to apply a function to each well with some default "factor" argument. We need a way to choose this factor globally, but we might sometimes need to adjust it for specific plates and even for specific wells within the plates. Essentially, we would like to copy the default factor and allow "overriding" of these copies. In Quibbler this is done quite seamlessly. See the example here.


* **Features**
    * Overriding assignments
    * Graphics-driven assignment
    * Inverse assignment
    * Assignment template


* **Try me**
    * Drag the horizontal line to the overall default "factor"
    * Drag the "per-plate factor" markers will prompt asking if assignments should be interpreted as overrides to the "plate-facor", or propagate upstream to change the default.    
    * Drag the "per-well-per-plate factor" markers will prompt asking if assignments should be interpreted as overrides a the wellplate level, or propagate one step up to the plate level or all the way up to the default level.    

In [1]:
from functools import partial
from pyquibbler import iquib, override_all, q
override_all()
import matplotlib.pyplot as plt
import numpy as np
%matplotlib tk

In [2]:
# user function to plot markers at specified colors
@partial(np.vectorize, excluded={0}, evaluate_now=True, pass_quibs=True)
def plot_dragger(ax, x, y, overridden):
    color = 'r' if overridden.get_value() else 'g'
    ax.plot(x, y, marker='d', markerfacecolor=color, markeredgecolor='None',
            markersize=12, linestyle='None', picker=True)

In [3]:
# Set number of plate and number wells per plate
n_plates = iquib(3)
n_wells = iquib(6)

In [4]:
# Figure setup
plt.figure()

ax1 = plt.subplot(2, 1, 1)
ax1.axis([-0.5, n_plates - 0.5, 0, 100])
ax1.set_ylabel('Plate factor')
ax1.set_xticks(np.arange(n_plates))

ax2 = plt.subplot(2, 1, 2)
ax2.axis([-0.5, n_plates - 0.5, 0, 100])
ax2.set_ylabel('Well factor');
ax2.set_xticks(np.arange(n_plates));

In [5]:
# Common properties
input_properties = {'assignment_template':(0, 100, 1), 'allow_overriding':True}

In [6]:
# Define and plot the default factor
default_factor = iquib(np.array([70])).setp(**input_properties, name='Default')
ax1.plot([-0.5, n_plates - 0.5], default_factor[[0, 0]], 'k', linewidth=5, picker=True);

In [7]:
# Define and plot the per-plate factor
plate_factor = np.repeat(default_factor, n_plates, 0).setp(**input_properties, name='Plate_factor')
x = np.arange(n_plates)
ax1.bar(x, plate_factor, color=(0.7, 0.7, 0.7))
plot_dragger(ax1, x, plate_factor, plate_factor.get_override_mask());

In [8]:
# Define the per-plate-per-well factor
well_factor = np.repeat(plate_factor, n_wells, 0).setp(**input_properties, name='Well_factor')
dd = np.linspace(-0.4, 0.4, n_wells + 1)
dd = (dd[0:-1] + dd[1:]) / 2.
xx = np.ravel(x + np.reshape(dd, (n_wells, 1)), 'F')
ax2.bar(xx, well_factor, color=(0.7, 0.7, 0.7), width=0.1)
plot_dragger(ax2, xx, well_factor, well_factor.get_override_mask());

In [9]:
plt.show()